In [1]:
import torch
from mlp import make_dataset, CharacterLevelMLP
import matplotlib.pyplot as plt
from torch.optim import Adam
from pathlib import Path
from torch.nn.functional import cross_entropy
%matplotlib inline

In [2]:
names_file = Path("./names.txt")
X , Y = make_dataset(names_file)

In [3]:
print(X.shape)

torch.Size([228146, 3])


In [4]:
EPOCHS = 10
GRAD_STEP = 0.1


# ================================

# C = torch.randn((27, 2))
# W1 = torch.randn((6, 100))
# b1 = torch.randn(100)
# W2 = torch.randn((100, 27))
# b2 = torch.randn(27)

# parameters = [C,W1,b1,W2,b2]

# for p in parameters:
#     p.requires_grad = True
    
# ================================
    
mlp_network = CharacterLevelMLP()

# Check if model has parameters
print("Model parameters:")
for name, param in mlp_network.named_parameters():
    print(f"{name}: {param.shape}")

adam = Adam(mlp_network.parameters(), lr=GRAD_STEP)
mlp_network.train()

# ================================

losses = []


for epoch in range(EPOCHS):
    # embeddings = C[X]
    # hidden_dim = embeddings.view((embeddings.shape[0], 6)) @ W1 + b1
    # logits = hidden_dim @ W2 + b2
    logits = mlp_network(X)
    loss = cross_entropy(logits, Y)
    losses.append(loss.item())
        
    # for p in parameters:
    #     p.grad = None
    # Clear gradients from previous iteration
    adam.zero_grad()
        
    loss.backward()
    
    # for p in parameters:
    #     p.data += -GRAD_STEP * p.grad
    # Update parameters
    adam.step()
    
    

Model parameters:
C: torch.Size([27, 2])
W1: torch.Size([6, 100])
b1: torch.Size([100])
W2: torch.Size([100, 27])
b2: torch.Size([27])


In [5]:
losses

[42.45361328125,
 23.07086181640625,
 16.27788734436035,
 11.91812515258789,
 10.330805778503418,
 10.45801067352295,
 11.431381225585938,
 11.014293670654297,
 9.66076946258545,
 8.822908401489258]